In [8]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

import os
os.chdir('..')

In [2]:
from langchain_zhipu import ChatZhipuAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document
from langchain.agents import tool

from textlong.agents import create_tools_calling_executor
from textlong.qa import AskDocumentTool, create_qa_chain

query_document = Document(page_content="textlong 是一个基于AI的开源框架，用于生成超长文档")

db = FAISS.from_documents([query_document], OpenAIEmbeddings())

llm_zhipu = ChatZhipuAI()

chain = create_qa_chain(llm_zhipu, db.as_retriever())
qa_tool = AskDocumentTool(chain=(chain | StrOutputParser()), name="qa_chain")

In [3]:
chain.invoke({"query": "textlong是什么"})

AIMessage(content='textlong是一个基于人工智能技术的开源框架，主要用于生成超长文档。通过这一框架，用户可以自动生成大量的文本内容，适用于需要大量写作和内容生成的场合。', response_metadata={'id': '8689069584141990710', 'created': 1716708686, 'token_usage': {'completion_tokens': 38, 'prompt_tokens': 52, 'total_tokens': 90}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-24021477-405b-45d7-bfbb-a3f919cf63fb-0')

In [4]:
chain.get_graph().print_ascii()

              +---------------------------------+           
              | Parallel<context,question>Input |           
              +---------------------------------+           
                     ****               ****                
                  ***                       ***             
                **                             ***          
      +-------------+                             **        
      | Lambda(...) |                              *        
      +-------------+                              *        
             *                                     *        
             *                                     *        
             *                                     *        
 +----------------------+                          *        
 | VectorStoreRetriever |                          *        
 +----------------------+                          *        
             *                                     *        
             *          

In [9]:
from langchain.tools import tool
import random

# @tool(args_schema=WhereIsCatSchema)
@tool("WhereIsCatHidding")
def where_is_cat_hiding(idea: str) -> str:
    """从这些地方选择猫躲藏的地方"""
    return random.choice(["在床底下", "在书架中", "在阳台"])

In [10]:
# from langchain_openai import ChatOpenAI
# llm_zhipu = ChatZhipuAI()
tools = [qa_tool, where_is_cat_hiding]
app = create_tools_calling_executor(llm_zhipu, tools)

In [7]:
for x in app.stream("讲个笑话给我?"):
    print(x)

{'agent': AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"讲个笑话给我"}', 'name': 'qa_chain'}, 'id': 'call_8689071473927619247', 'index': 0, 'type': 'function'}]}, response_metadata={'id': '8689071473927619247', 'created': 1716707721, 'token_usage': {'completion_tokens': 17, 'prompt_tokens': 243, 'total_tokens': 260}, 'model_name': 'glm-4', 'finish_reason': 'tool_calls'}, id='run-f4d9ca5d-7a3d-4a9f-91bc-7590d6b3ff2c-0', tool_calls=[{'name': 'qa_chain', 'args': {'query': '讲个笑话给我'}, 'id': 'call_8689071473927619247'}])}
{'action': [ToolMessage(content='对不起，我作为一个基于文本生成的AI模型，主要目的是提供信息和解答相关问题。而我所依赖的资料中，并没有包含娱乐性质的内容，如笑话。如果你有关于textlong这个开源框架或者其他相关技术问题，我会很乐意为你提供帮助。', tool_call_id='call_8689071473927619247')]}
{'agent': AIMessage(content='抱歉，我无法提供笑话，但我可以帮你解答问题和提供信息。有什么我可以帮助你的吗？', response_metadata={'id': '8689076043773076945', 'created': 1716707727, 'token_usage': {'completion_tokens': 23, 'prompt_tokens': 315, 'total_tokens': 338}, 'model_name': 'glm-4',

In [11]:
from langchain_core.messages import HumanMessage
app = create_tools_calling_executor(
    llm_zhipu,
    tools = tools,
    verbose = True)

input = "请查询资料，告诉我textlong是什么？"
async for chunk in app.astream_events(input, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])
    elif(chunk['event']=="on_chat_model_start"):
	    print("\n", chunk['event'], chunk['name'], chunk['tags'])        


 on_chat_model_start ChatZhipuAI ['seq:step:2']
__runnable-name:  qa_chain
log: call tool [qa_chain]
{}
{'chunk': [ToolMessage(content='textlong 是一个基于人工智能技术的开源框架，它主要用于生成超长文档。通过这一框架，用户可以自动化地生成大量的文本内容，适用于需要大量写作和文档生成的场景。', tool_call_id='call_8689071370848456029')]}
{'input': ['请查询资料，告诉我textlong是什么？', AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'id': 'call_8689071370848456029', 'index': 0, 'type': 'function', 'function': {'name': 'qa_chain', 'arguments': '{"query":"textlong是什么"}'}}]}, response_metadata={'finish_reason': 'tool_callstool_calls', 'model': 'glm-4glm-4', 'created': 1716713663, 'index': 0, 'usage': {'prompt_tokens': 247, 'completion_tokens': 16, 'total_tokens': 263}}, id='8689071370848456029', tool_calls=[{'name': 'qa_chain', 'args': {'query': 'textlong是什么'}, 'id': 'call_8689071370848456029'}], tool_call_chunks=[{'name': 'qa_chain', 'args': '{"query":"textlong是什么"}', 'id': 'call_8689071370848456029', 'index': 0}])], 'output': [ToolMessage(content='textlong 是一个

In [12]:
app.get_graph().print_ascii()

                    +-----------+             
                    | __start__ |             
                    +-----------+             
                          *                   
                          *                   
                          *                   
                      +-------+               
                      | agent |               
                     *+-------+*              
                   **           ***           
                 **                **         
               **                    **       
+-----------------------+              **     
| agent_should_continue |               *     
+-----------------------+               *     
            *           *****           *     
            *                ****       *     
            *                    ***    *     
       +---------+                +--------+  
       | __end__ |                | action |  
       +---------+                +--------+  


In [13]:
app = create_tools_calling_executor(
    llm_zhipu,
    tools = tools,
    runnables = {"qa_chain": chain},
    verbose = True
)

input = ["请查询资料，告诉我textlong是什么？"]
async for chunk in app.astream_events(input, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])
    elif(chunk['event']=="on_chat_model_start"):
	    print("\n", chunk['event'], chunk['name'], chunk['tags'])


 on_chat_model_start ChatZhipuAI ['seq:step:2']
__runnable-name:  qa_chain
log: call runnable [qa_chain]


TypeError: list indices must be integers or slices, not str

In [23]:
app.get_graph().print_ascii()

                        +-----------+                     
                        | __start__ |                     
                        +-----------+                     
                              *                           
                              *                           
                              *                           
                          +-------+                       
                          | agent |*                      
                         *+-------+ ****                  
                       **               ***               
                     **                    ***            
                   **                         ****        
    +-----------------------+                     **      
    | agent_should_continue |                      *      
    +-----------------------+**                    *      
           **        **        *******             *      
         **            **             *****        *    

In [8]:
app = create_tools_calling_executor(
    llm_zhipu,
    tools = tools,
    runnables = {"qa_chain": {"node": chain, "to": "agent"}},
    verbose = True)
app.get_graph().print_ascii()

                              +-----------+                          
                              | __start__ |                          
                              +-----------+                          
                                     *                               
                                     *                               
                                     *                               
                                +-------+                            
                               *| agent |***                         
                            *** +-------+   ***                      
                       *****          *        *****                 
                    ***                *            ***              
                 ***                   *               *****         
+-----------------------+               *                   **       
| agent_should_continue |*             *                     *       
+-------------------

In [9]:
inputs = ["请查询资料，告诉我langchain_chinese是什么？"]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])
    elif(chunk['event']=="on_chat_model_start"):
	    print("\n", chunk['event'], chunk['name'], chunk['tags'])


 on_chat_model_start ChatZhipuAI ['seq:step:2']
__runnable-name:  qa_chain
log: call runnable [qa_chain]


Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1



 on_chat_model_start ChatZhipuAI ['seq:step:4']
lang_chain__ch_inese_ 是_专门_针对_中国_大_语言_模型_进行_优化_和_改进_的_lang_chain_模块_。_它_可能是_lang_chain_这一_框架_的_扩展_或_衍生_版本_，_旨在_更好地_适应_中文_语言_环境_下的_需求_，_提升_大_语言_模型_在_中文_处理_方面的_性能_和_效果_。__
 on_chat_model_start ChatZhipuAI ['seq:step:2']
根据_我的_查询_结果_，_lang_chain__ch_inese_ 是_专门_针对_中国_大_语言_模型_进行_优化_和_改进_的_ lang_chain_ 模_块_。_它是_ lang_chain_ 这一_框架_的_扩展_或_衍生_版本_，_旨在_更好地_适应_中文_语言_环境_下的_需求_，_提升_大_语言_模型_在_中文_处理_方面的_性能_和_效果_。__

In [10]:
inputs = [HumanMessage(content="猫在哪里？")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

__runnable-name:  WhereIsCatHidding
log: call tool [WhereIsCatHidding]
{}
{'chunk': [ToolMessage(content='在床底下', tool_call_id='call_8483752349020160968')]}
{'input': [HumanMessage(content='猫在哪里？'), AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8483752349020160968', 'function': {'arguments': '{"idea":"猫在哪里"}', 'name': 'WhereIsCatHidding'}, 'type': 'function'}]})], 'output': [ToolMessage(content='在床底下', tool_call_id='call_8483752349020160968')]}
根据_我的_观察_和_经验_，_猫_通常会_躲_在一些_狭_小_的地方_，_比如_床_底下_、_沙发_后面_或者_柜_子里_。_所以_，_如果你想_找到_一只_猫_，_最好_在这些_地方_仔细_寻找_。__

In [25]:
inputs =  [HumanMessage(content="霍金的生日是哪一天？")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="agent"):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

__runnable-name:  qa_chain
log: call runnable [qa_chain]


In [11]:
inputs =  [HumanMessage(content="写一个关于程序员的笑话")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="agent"):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

为什么_程序员_总是_携带_电脑_？_因为他们_不想_被人_称为_“_裸_奔_者_”。__